In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asba-nlim/airy_test_nlim.csv
/kaggle/input/asba-nlim/airy_test_nlim2.csv
/kaggle/input/asba-nlim/rest_test_nlim.csv
/kaggle/input/asba-nlim/car_train_nlim.csv
/kaggle/input/asba-nlim/rest_train_nlim.csv
/kaggle/input/asba-nlim/airy_train_nlim.csv
/kaggle/input/asba-nlim/car_test_nlim.csv
/kaggle/input/sentence-piece-model-airy-only/airy_sp10m.cased.v3.model
/kaggle/input/sentence-piece-model-airy-only/airy_corpus.txt
/kaggle/input/sentence-piece-model-airy-only/airy_sp10m.cased.v3.vocab
/kaggle/input/corpus-sentence-piece-asba/airy_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-asba/airy_rest_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-asba/car_sp10m.cased.v3.vocab
/kaggle/input/corpus-sentence-piece-asba/car_rest_sp10m.cased.v3.vocab
/kaggle/input/corpus-sentence-piece-asba/car_rest_corpus.txt
/kaggle/input/corpus-sentence-piece-asba/airy_corpus.txt
/kaggle/input/corpus-sentence-piece-asba/airy_car_sp10m.cased.v3.model
/kaggle/input/corpus-sentence

In [2]:
import numpy as np
import os
import pandas as pd

In [3]:
path_to_data='/kaggle/input/corpus-sentence-piece-asba'
# os.listdir(path_to_data)###

In [4]:
print (os.listdir(path_to_data))

['airy_sp10m.cased.v3.model', 'airy_rest_sp10m.cased.v3.model', 'car_sp10m.cased.v3.vocab', 'car_rest_sp10m.cased.v3.vocab', 'car_rest_corpus.txt', 'airy_corpus.txt', 'airy_car_sp10m.cased.v3.model', 'rest_corpus.txt', 'all_sp10m.cased.v3.model', 'car_rest_sp10m.cased.v3.model', 'airy_rest_corpus.txt', 'rest_sp10m.cased.v3.vocab', 'airy_sp10m.cased.v3.vocab', 'all_sp10m.cased.v3.vocab', 'airy_car_sp10m.cased.v3.vocab', 'airy_rest_sp10m.cased.v3.vocab', 'all_corpus.txt', 'rest_sp10m.cased.v3.model', 'car_corpus.txt', 'car_sp10m.cased.v3.model']


In [5]:
##clone xlnet official repository
!git clone https://github.com/arfaghif/malaya

Cloning into 'malaya'...
remote: Enumerating objects: 17730, done.
remote: Counting objects: 100% (17730/17730), done.
remote: Compressing objects: 100% (4511/4511), done.
remote: Total 17730 (delta 12831), reused 17712 (delta 12823), pack-reused 0
Receiving objects: 100% (17730/17730), 122.06 MiB | 28.18 MiB/s, done.
Resolving deltas: 100% (12831/12831), done.


### Using sentence piece model from Google to tokenize text
We will be using  Google's **sentencepiece** model to tokenize the text. 

In [6]:
!pip install sentencepiece

In [7]:
path_to_data

'/kaggle/input/corpus-sentence-piece-asba'

In [8]:
curr_dir=os.getcwd()

In [9]:
curr_dir

'/kaggle/working'

In [10]:
%cd '/kaggle/input/corpus-sentence-piece-asba'


/kaggle/input/corpus-sentence-piece-asba


In [11]:
%cd '/kaggle/working/malaya/pretrained-model/xlnet'

/kaggle/working/malaya/pretrained-model/xlnet


In [12]:
print (os.listdir(path_to_data))

['airy_sp10m.cased.v3.model', 'airy_rest_sp10m.cased.v3.model', 'car_sp10m.cased.v3.vocab', 'car_rest_sp10m.cased.v3.vocab', 'car_rest_corpus.txt', 'airy_corpus.txt', 'airy_car_sp10m.cased.v3.model', 'rest_corpus.txt', 'all_sp10m.cased.v3.model', 'car_rest_sp10m.cased.v3.model', 'airy_rest_corpus.txt', 'rest_sp10m.cased.v3.vocab', 'airy_sp10m.cased.v3.vocab', 'all_sp10m.cased.v3.vocab', 'airy_car_sp10m.cased.v3.vocab', 'airy_rest_sp10m.cased.v3.vocab', 'all_corpus.txt', 'rest_sp10m.cased.v3.model', 'car_corpus.txt', 'car_sp10m.cased.v3.model']


In [13]:
##We will be saving tf records in a folder named experimenttfrecords

In [14]:
# !git pull origin versi3.7

In [15]:
%%writefile prepro_utils.py
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import unicodedata
import six
from functools import partial


SPIECE_UNDERLINE = '▁'


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def print_(*args):
  new_args = []
  for arg in args:
    if isinstance(arg, list):
      s = [printable_text(i) for i in arg]
      s = ' '.join(s)
      new_args.append(s)
    else:
      new_args.append(printable_text(arg))
  print(*new_args)


def preprocess_text(inputs, lower=False, remove_space=True, keep_accents=False):
  if remove_space:
    outputs = ' '.join(inputs.strip().split())
  else:
    outputs = inputs
  outputs = outputs.replace("``", '"').replace("''", '"')

  if six.PY2 and isinstance(outputs, str):
    outputs = outputs.decode('utf-8')

  if not keep_accents:
    outputs = unicodedata.normalize('NFKD', outputs)
    outputs = ''.join([c for c in outputs if not unicodedata.combining(c)])
  if lower:
    outputs = outputs.lower()

  return outputs


def encode_pieces(sp_model, text, return_unicode=True, sample=False):
  # return_unicode is used only for py2

  # note(zhiliny): in some systems, sentencepiece only accepts str for py2
  if six.PY2 and isinstance(text, unicode):
    text = text.encode('utf-8')

  if not sample:
    pieces = sp_model.EncodeAsPieces(text)
  else:
    pieces = sp_model.SampleEncodeAsPieces(text, 64, 0.1)
  new_pieces = []
  for piece in pieces:
    if len(piece) > 1 and piece[-1] == ',' and piece[-2].isdigit():
      cur_pieces = sp_model.EncodeAsPieces(
          piece[:-1].replace(SPIECE_UNDERLINE, ''))
      if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
        if len(cur_pieces[0]) == 1:
          cur_pieces = cur_pieces[1:]
        else:
          cur_pieces[0] = cur_pieces[0][1:]
      cur_pieces.append(piece[-1])
      new_pieces.extend(cur_pieces)
    else:
      new_pieces.append(piece)

  # note(zhiliny): convert back to unicode for py2
  if six.PY2 and return_unicode:
    ret_pieces = []
    for piece in new_pieces:
      if isinstance(piece, str):
        piece = piece.decode('utf-8')
      ret_pieces.append(piece)
    new_pieces = ret_pieces

  return new_pieces


def encode_ids(sp_model, text, sample=False):
  pieces = encode_pieces(sp_model, text, return_unicode=False, sample=sample)
  ids = [sp_model.PieceToId(piece) for piece in pieces]
  return ids


if __name__ == '__main__':
  import sentencepiece as spm

  sp = spm.SentencePieceProcessor()
  sp.load('sp10m.uncased.v3.model')

  print_(u'I was born in 2000, and this is falsé.')
  print_(u'ORIGINAL', sp.EncodeAsPieces(u'I was born in 2000, and this is falsé.'))
  print_(u'OURS', encode_pieces(sp, u'I was born in 2000, and this is falsé.'))
  print(encode_ids(sp, u'I was born in 2000, and this is falsé.'))
  print_('')
  prepro_func = partial(preprocess_text, lower=True)
  print_(prepro_func('I was born in 2000, and this is falsé.'))
  print_('ORIGINAL', sp.EncodeAsPieces(prepro_func('I was born in 2000, and this is falsé.')))
  print_('OURS', encode_pieces(sp, prepro_func('I was born in 2000, and this is falsé.')))
  print(encode_ids(sp, prepro_func('I was born in 2000, and this is falsé.')))
  print_('')
  print_('I was born in 2000, and this is falsé.')
  print_('ORIGINAL', sp.EncodeAsPieces('I was born in 2000, and this is falsé.'))
  print_('OURS', encode_pieces(sp, 'I was born in 2000, and this is falsé.'))
  print(encode_ids(sp, 'I was born in 2000, and this is falsé.'))
  print_('')
  print_('I was born in 92000, and this is falsé.')
  print_('ORIGINAL', sp.EncodeAsPieces('I was born in 92000, and this is falsé.'))
  print_('OURS', encode_pieces(sp, 'I was born in 92000, and this is falsé.'))
  print(encode_ids(sp, 'I was born in 92000, and this is falsé.'))

Overwriting prepro_utils.py


In [16]:
!python data_utils.py \
   --use_tpu=False \
   --bsz_per_host=8 \
   --num_core_per_host=1 \
   --uncased=False \
   --seq_len=128 \
   --reuse_len=64 \
   --input_glob="/kaggle/input/corpus-sentence-piece-asba/car_corpus.txt" \
   --save_dir="/kaggle/working/c_experimenttfrecords" \
   --sp_path="/kaggle/input/corpus-sentence-piece-asba/car_sp10m.cased.v3.model" \
   --mask_alpha=6 --mask_beta=1 --num_predict=50 --bi_data=False

I0227 10:56:29.774576 139769993086784 data_utils.py:211] Use glob: /kaggle/input/corpus-sentence-piece-asba/car_corpus.txt
I0227 10:56:29.775012 139769993086784 data_utils.py:212] Find 1 files: ['/kaggle/input/corpus-sentence-piece-asba/car_corpus.txt']
I0227 10:56:29.775184 139769993086784 data_utils.py:220] Task 0 process 1 files: ['/kaggle/input/corpus-sentence-piece-asba/car_corpus.txt']
I0227 10:56:29.790243 139769993086784 data_utils.py:89] Processing /kaggle/input/corpus-sentence-piece-asba/car_corpus.txt
I0227 10:56:29.794114 139769993086784 data_utils.py:92] Loading line 0
I0227 10:56:29.907383 139769993086784 data_utils.py:112] Finish with line 2380
data_utils.py:117: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://n

Now, it's time to train the model.

In [17]:
!wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/xlnet-base-29-03-2020.tar.gz

--2023-02-27 10:56:32--  https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/xlnet-base-29-03-2020.tar.gz
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1275743608 (1.2G) [application/octet-stream]
Saving to: ‘xlnet-base-29-03-2020.tar.gz’

xlnet-base-29-03-20 100%[===================>]   1.19G  31.4MB/s    in 42s     

2023-02-27 10:57:14 (29.1 MB/s) - ‘xlnet-base-29-03-2020.tar.gz’ saved [1275743608/1275743608]



In [18]:
!tar -xvf xlnet-base-29-03-2020.tar.gz

xlnet-base-29-03-2020/
xlnet-base-29-03-2020/sp10m.cased.v9.vocab
xlnet-base-29-03-2020/sp10m.cased.v9.model
xlnet-base-29-03-2020/config.json
xlnet-base-29-03-2020/model.ckpt-300000.data-00000-of-00001
xlnet-base-29-03-2020/model.ckpt-300000.index
xlnet-base-29-03-2020/model.ckpt-300000.meta


###  We will be training our language model from checkpoint. Checkpoint is the model downloaded above.


In [19]:
!python train_gpu.py \
   --corpus_info_path="/kaggle/working/c_experimenttfrecords/corpus_info.json" \
   --model_dir="/kaggle/working/c_experimenttfrecords/model" \
   --record_info_dir="/kaggle/working/c_experimenttfrecords/tfrecords" \
   --train_batch_size=8 \
   --seq_len=128 \
   --reuse_len=64 \
   --mem_len=384 \
   --perm_size=64 \
   --untie_r=True \
   --mask_alpha=6 \
   --mask_beta=1 \
   --num_predict=50 \
   --train_steps=40 \
   --iterations=10 \
   --uncased=False \
   --bi_data=False \
   --save_steps=2 \
   --d_head=64 \
   --d_inner=3072 \
   --d_model=768 \
   --ff_activation=gelu \
   --n_head=12 \
   --n_layer=12 \
   --n_token=32000 \
   --init_checkpoint='xlnet-base-29-03-2020/model.ckpt-300000'

I0227 10:57:34.271095 140454916200256 train_gpu.py:319] n_token 32000
I0227 10:57:34.272439 140454916200256 data_utils.py:797] Use the following tfrecord dirs: ['/kaggle/working/c_experimenttfrecords/tfrecords']
I0227 10:57:34.272913 140454916200256 data_utils.py:801] [0] Record glob: /kaggle/working/c_experimenttfrecords/tfrecords/record_info-train-*.bsz-8.seqlen-128.reuse-64.uni.alpha-6.beta-1.fnp-50.json
I0227 10:57:34.274034 140454916200256 data_utils.py:805] [0] Num of record info path: 1
I0227 10:57:34.274428 140454916200256 data_utils.py:838] [Dir 0] Number of chosen batches: 56
I0227 10:57:34.274559 140454916200256 data_utils.py:840] [Dir 0] Number of chosen files: 1
I0227 10:57:34.274665 140454916200256 data_utils.py:841] ['/kaggle/working/c_experimenttfrecords/tfrecords/train-0-0.bsz-8.seqlen-128.reuse-64.uni.alpha-6.beta-1.fnp-50.tfrecords']
I0227 10:57:34.274778 140454916200256 data_utils.py:848] Total number of batches: 56
I0227 10:57:34.275002 140454916200256 data_utils.p

In [20]:
%cd '/kaggle/working'

/kaggle/working


In [21]:
!mkdir '/kaggle/working/c_experimenttfrecords/model/c_pytorch_model'

In [22]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 132773, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 132773 (delta 222), reused 301 (delta 188), pack-reused 132350
Receiving objects: 100% (132773/132773), 127.32 MiB | 27.15 MiB/s, done.
Resolving deltas: 100% (100423/100423), done.


Converting tensorflow model into pytorch model

In [23]:
%cd 'transformers'

/kaggle/working/transformers


In [24]:
# !export TRANSFO_XL_CHECKPOINT_PATH='/content/drive/My Drive/experimenttfrecords/model/model.ckpt'
# !export TRANSFO_XL_CONFIG_PATH='/content/drive/My Drive/experimenttfrecords/model/config.json'
# !export PYTORCH_MODELPATH='/content/drive/My Drive/experimenttfrecords/pytorch_model'

In [25]:
!pip install transformers

In [26]:
!transformers-cli convert \
  --model_type xlnet  \
  --tf_checkpoint="/kaggle/working/c_experimenttfrecords/model/model.ckpt" \
  --config="/kaggle/working/c_experimenttfrecords/model/config.json" \
  --pytorch_dump_output="/kaggle/working/c_experimenttfrecords/model/c_pytorch_model"

Loading TF weight beta1_power with shape []
Loading TF weight beta2_power with shape []
Loading TF weight global_step with shape []
Loading TF weight model/lm_loss/bias with shape [32000]
Loading TF weight model/lm_loss/bias/Adam with shape [32000]
Loading TF weight model/lm_loss/bias/Adam_1 with shape [32000]
Loading TF weight model/transformer/layer_0/ff/layer_1/bias with shape [3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/bias/Adam with shape [3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/bias/Adam_1 with shape [3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/kernel with shape [768, 3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/kernel/Adam with shape [768, 3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/kernel/Adam_1 with shape [768, 3072]
Loading TF weight model/transformer/layer_0/ff/layer_2/bias with shape [768]
Loading TF weight model/transformer/layer_0/ff/layer_2/bias/Adam with shape [768]
Loading TF weigh

In [27]:
!rm -r "/kaggle/working/transformers"
!rm -r "/kaggle/working/malaya"

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
